### Logistic Regression

#### Import Packages

In [43]:
%run ml_helpers.ipynb
%run ml.ipynb

               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
1035442  11503220     False   914  2018    11    45   10     4   First  0.00   
1154083  11677723     False  2423  2019     5    18    5    13  Second  0.00   
1041107  11511105     False   112  2018    11    46   18    17   Third  0.10   
1103877  11673922     False  1634  2019     2     8   20    13  Second  0.06   
1276675  11857737     False  1414  2019    10    41   10    19   Third  0.07   
1168139  11700233     False  1915  2019     5    21   24     5   First  0.09   
807027   11197609     False   422  2018     1     1    7    12  Second  0.05   
1049569  11522405     False   524  2018    12    48    1     9  Second  0.78   
1263845  11837656     False   321  2019     9    39   23     2   First  0.00   
1017608  11479623      True  1011  2018    10    42   17    14  Second  0.00   
1105886  11604673     False   922  2019     2     8   23    11  Second  0.45   
902775   11330827     False  1831  2018 

On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
               ID  Domestic  Beat  Year Month  Week  Day  Hour   Watch  PRCP  \
828349   11226749     False  1912  2018     2     6    9    12  Second  0.64   
1271009  11848506      True  1011  2019    10    40    2    18   Third  0.57   
1322531  11929336     False  1922  2019    12    51   17    16   Third  0.00   
939155   11374568     False  1123  2018     7    28    9    11  Second  0.00   
965879   11410257     False   822  2018     8    32   11     6   First  0.00   
1165169  11693224      True   233  2019     5    21   20 

         Beat  Year  Month   Watch  PRCP  SNOW  TMAX  TMIN  count_l_stops  \
1084459   712  2019      1   Third  0.34   5.0    29    15            0.0   
1306908   124  2019     11  Second  0.00   0.0    50    33            0.0   
1259902   734  2019      9   Third  0.00   0.0    79    66            0.0   
1130490   211  2019      4   Third  0.00   0.0    49    27            1.0   
1223435  2412  2019      8   Third  0.00   0.0    80    60            0.0   
...       ...   ...    ...     ...   ...   ...   ...   ...            ...   
1134432   323  2019      4   Third  0.16   0.0    70    53            0.0   
1117840   724  2019      3  Second  0.49   0.0    64    40            0.0   
1239610  1232  2019      8   First  0.00   0.0    78    69            1.0   
1275167   423  2019     10   Third  0.00   0.0    71    48            0.0   
1102334   423  2019      2   Third  0.13   2.1    29    25            0.0   

         count_bus_stops  ...  count_restaurants  count_bars  count_daycare

Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot 

In [47]:
import pandas as pd
import geopandas as gpd
import seaborn as sns
import numpy as np
import sklearn as sk
from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import precision_score
import datetime

#### Read and Process Data

In [48]:
data = pd.read_csv("../intermediate_data/high_crime_labeled.csv")
data = data.drop(columns=["beat", "beat_num", "Crimes", "Arrest", "Domestic", "Serious"])
data = convert_to_categorical(data, ["district", "sector",
                                    "Month",
                                     "Watch", "Beat"])
data.drop(columns = ["Unnamed: 0"], inplace=True)

In [49]:
data.dtypes

district               category
sector                 category
Year                      int64
Month                  category
Watch                  category
Beat                   category
count_l_stops           float64
count_bus_stops         float64
count_metra_stops       float64
count_restaurants       float64
count_bars              float64
count_daycares          float64
count_entertainment     float64
count_businesses        float64
road_distance_ft        float64
TOTAL POPULATION        float64
dist_to_police          float64
dist_to_hospital        float64
SNOW                    float64
TMAX                    float64
TMIN                    float64
high_crime              float64
high_crime_geog_lag     float64
dtype: object

#### Setup ML

In [50]:
models = {
    'LogisticRegression': LogisticRegression(), 
}
grid = {
    'LogisticRegression': [{'penalty': x, 'C': y, 'random_state': 0} 
                           for x in ('l2', 'none') \
                           for y in (0.01, 0.1, 1, 10, 100)]
}

In [51]:
data_list = prep_data(data, "high_crime",
                                        2, "Year", ["district", "sector", 
                                                    "Month", 
                                                    "Beat", "Watch"])

      district sector  Year Month   Watch  Beat  count_l_stops  \
29557       17      1  2018     1   First  1713            3.0   
29558       17      1  2018     1  Second  1713            3.0   
29559       17      1  2018     1   Third  1713            3.0   
29560       17      1  2018     2   First  1713            3.0   
29561       17      1  2018     2  Second  1713            3.0   
...        ...    ...   ...   ...     ...   ...            ...   
49256        3      1  2019    11  Second   312            2.0   
49257        3      1  2019    11   Third   312            2.0   
49258        3      1  2019    12   First   312            2.0   
49259        3      1  2019    12  Second   312            2.0   
49260        3      1  2019    12   Third   312            2.0   

       count_bus_stops  count_metra_stops  count_restaurants  ...  \
29557             26.0                0.0               63.0  ...   
29558             26.0                0.0               63.0  ...   


Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2016 2017]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...
Working on: [2015 2016]
Have accessed train and test df...
Finished filling NAs with mean...
On to normalizing continuous...
Finished normalizing training data
Finished normalizing test data...
Finished one-hot encoding...
Finished one-hot encoding...
Finished standardizing...


In [52]:
logistic_results = average_grid_searches(LogisticRegression(), "LogisticRegression", 
                          grid, 
                          data_list, "high_crime", 2020)

test year is: 2020
[2016, 2017, 2018, 2019]


In [53]:
logistic_results

,parameters_2019,precision_2019,parameters_2018,precision_2018,parameters_2017,precision_2017,mean
0,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.736949,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.731973,"{'penalty': 'l2', 'C': 0.01, 'random_state': 0}",0.728563,0.732495
0,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.723355,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.730464,"{'penalty': 'l2', 'C': 1, 'random_state': 0}",0.715122,0.722980
0,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 0.01, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 0.1, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 1, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 10, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.719931,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.719673,"{'penalty': 'none', 'C': 100, 'random_state': 0}",0.715348,0.718317
0,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.721973,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.707954,"{'penalty': 'l2', 'C': 100, 'random_state': 0}",0.717263,0.715730
0,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.725150,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.721084,"{'penalty': 'l2', 'C': 10, 'random_state': 0}",0.698679,0.714971
0,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.690270,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.725776,"{'penalty': 'l2', 'C': 0.1, 'random_state': 0}",0.718531,0.711526
